# Debugging with Evidently Test Suites and Reports

In [ ]:
import datetime
import pandas as pd

from evidently import DataDefinition
from evidently import Dataset
from evidently import Report
from evidently.presets import DataDriftPreset

from joblib import dump, load

from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error

## Load data and model

In [ ]:
ref_data = pd.read_parquet("data/reference.parquet")

In [ ]:
current_data = pd.read_parquet("data/green_tripdata_2022-02.parquet")

In [ ]:
with open("models/lin_reg.bin", "rb") as f_in:
    model = load(f_in)

In [ ]:
# data labeling
target = "duration_min"
num_features = ["passenger_count", "trip_distance", "fare_amount", "total_amount"]
cat_features = ["PULocationID", "DOLocationID"]

In [ ]:
problematic_data = current_data.loc[
    (current_data.lpep_pickup_datetime >= datetime.datetime(2022, 2, 2, 0, 0))
    & (current_data.lpep_pickup_datetime < datetime.datetime(2022, 2, 3, 0, 0))
]

## Generate Report with Tests (ex Test Suite)

### Note

Since Evidently 0.7.0 release, Test Suite is integrated into reports.

To include tests into report, you just past tests to metric or provide `include_tests=True` to include default tests (if provided).


In [ ]:
data_definition = DataDefinition(
    numerical_columns=num_features + ["prediction"],
    categorical_columns=cat_features,
)

In [ ]:
problematic_data["prediction"] = model.predict(
    problematic_data[num_features + cat_features].fillna(0)
)

In [ ]:
report = Report([DataDriftPreset()], include_tests=True)

ref_dataset = Dataset.from_pandas(ref_data, data_definition=data_definition)
problematic_dataset = Dataset.from_pandas(ref_data, data_definition=data_definition)


run = report.run(reference_data=ref_dataset, current_data=problematic_dataset)

In [ ]:
run